In [90]:
#转换思路，用LSTM学习人类曲线后去预判，然后把LOSS当特征
import numpy as np
from sklearn.preprocessing import normalize
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [67]:
def get_track(s):
    i,moves,des,flag = s.split(' ')
    moves=moves.split(';')[:-1]
    moves=[move.split(',') for move in moves]
    moves=[[int(ll) for ll in l] for l in moves]
    des=[float(p) for p in des.split(',')]
    flag=int(flag)
#     print(i,moves[0],'->',des,flag)
    m = np.matrix(moves)
    m = m[np.lexsort(m.T)][0]
#     print(m)
    return (i,(m[0,0],m[0,1]), m, des, flag)

In [ ]:
people = []
machine = []
test = []
with open('dsjtzs_txfz_training.txt') as f:
    people = list(filter(lambda l:l.strip().endswith('1'),f.readlines()))
    f.seek(0)
    machine = list(filter(lambda l:l.strip().endswith('0'),f.readlines()))
with open('dsjtzs_txfz_test1.txt') as f:
    test = f.readlines()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

model = Sequential()
model.add(Conv1D(2, 2, activation='relu', input_shape=(None, 3)))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# model.fit(x_train, y_train, batch_size=16, epochs=10)
# score = model.evaluate(x_test, y_test, batch_size=16)

In [112]:
dataS = people
dataA = np.empty(len(dataS),dtype=np.object)
for i,d in enumerate(dataS):
    index, start, track, des, flag = get_track(d)
    dataA[i] = track
    

In [93]:
X_train = dataA
Y_train = np.ones(dataA.shape[0]).reshape((-1,1))
print(dataA[0].shape)

(194, 3)


In [ ]:
model.fit(np.expand_dims(X_train[0], axis=2), Y_train[0], batch_size=1, epochs=1)

In [154]:
class LSTMTagger(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.lstm = nn.LSTM(input_dim, hidden_dim)
        
        # The linear layer that maps from hidden state space to tag space
        self.hidden2pos = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))
        
    def forward(self, track):
        lstm_out, self.hidden = self.lstm(track.view(len(track), 1, -1), self.hidden)
        pos_space = self.hidden2pos(lstm_out.view(len(track), -1))
        return pos_space

In [152]:
model = LSTMTagger(1, 10, 2)
loss_function = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [91]:
tensor = torch.LongTensor(X_train[0])
inputs = torch.LongTensor(X_train[0][:,2])
targets = torch.LongTensor(X_train[0][:,0:2])
# print(tensor.size(), targets.size())
# print(X_train[0])
# (targets.contiguous()).view(-1,2)

In [153]:
for epoch in range(2): # again, normally you would NOT do 300 epochs, it is toy data
    for i,tracks in enumerate(X_train):
        
        if tracks.shape[0] <=4:
            continue
        # Step 1. Remember that Pytorch accumulates gradients.  We need to clear them out
        # before each instance
        model.zero_grad()
        tracks = tracks - tracks[0]
        tracks = normalize(tracks, axis=0)

        # Also, we need to clear out the hidden state of the LSTM, detaching it from its
        # history on the last instance.
        model.hidden = model.init_hidden()
        for j in range(3, tracks.shape[0]):
            model.zero_grad()
            track = tracks[j-3:j]#已知前三点的序列

            # Step 2. Get our inputs ready for the network, that is, turn them into Variables
            # of word indices.
    #         print(tracks)
    #         print(tracks[0:-1,:].shape, tracks[1:,-2].shape)
            inputs = track
            targets = tracks[j, 0:2]
    #         print(inputs)
    #         print(targets)
    #         print(tracks)
    #         tensor = torch.Tensor(tracks)
            inputs = autograd.Variable(torch.Tensor(inputs)).contiguous()
            targets = autograd.Variable(torch.Tensor(targets)).contiguous()

            # Step 3. Run our forward pass.
            tag_scores = model(inputs.float())
            
    #         print(inputs)
    #         print(tag_scores)
    #         break
    #         print(tag_scores, targets)
    #         break
            # Step 4. Compute the loss, gradients, and update the parameters by calling
            # optimizer.step()
            loss = loss_function(tag_scores, targets.float())
            loss.backward(retain_variables=True)
            optimizer.step()
            
        if i%50 == 0:
            print(loss)

/home/miller/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


RuntimeError: size mismatch, m1: [6 x 1], m2: [6 x 2] at /b/wheel/pytorch-src/torch/lib/TH/generic/THTensorMath.c:1237

In [140]:
inputs.size()

torch.Size([193, 4])

In [115]:
import pandas as pd
import numpy as np
import random
from keras.models import Sequential, model_from_json
from keras.layers import Dense, LSTM, Dropout

class NeuralNetwork():
    def __init__(self, **kwargs):
        """
        :param **kwargs: output_dim=4: output dimension of LSTM layer; 
        activation_lstm='tanh': activation function for LSTM layers; 
        activation_dense='relu': activation function for Dense layer; 
        activation_last='sigmoid': activation function for last layer; 
        drop_out=0.2: fraction of input units to drop; 
        np_epoch=10, the number of epoches to train the model. 
            epoch is one forward pass and one backward pass of all the training examples; 
        batch_size=32: number of samples per gradient update. 
            The higher the batch size, the more memory space you'll need; 
        loss='mean_square_error': loss function; 
        optimizer='rmsprop'
        """
        self.output_dim = kwargs.get('output_dim', 8)
        self.activation_lstm = kwargs.get('activation_lstm', 'relu')
        self.activation_dense = kwargs.get('activation_dense', 'relu')
        self.activation_last = kwargs.get('activation_last', 'softmax')    # softmax for multiple output
        self.dense_layer = kwargs.get('dense_layer', 2)     # at least 2 layers
        self.lstm_layer = kwargs.get('lstm_layer', 2)
        self.drop_out = kwargs.get('drop_out', 0.2)
        self.nb_epoch = kwargs.get('nb_epoch', 10)
        self.batch_size = kwargs.get('batch_size', 100)
        self.loss = kwargs.get('loss', 'categorical_crossentropy')
        self.optimizer = kwargs.get('optimizer', 'rmsprop')

    def NN_model(self, trainX, trainY, testX, testY):
        """
        :param trainX: training data set
        :param trainY: expect value of training data
        :param testX: test data set
        :param testY: epect value of test data
        :return: model after training
        """
        print("Training model is LSTM network!")
        input_dim = trainX[1].shape[1]
        output_dim = trainY.shape[1] # one-hot label
        # print predefined parameters of current model:
        model = Sequential()
        # applying a LSTM layer with x dim output and y dim input. Use dropout parameter to avoid overfitting
        model.add(LSTM(output_dim=self.output_dim,
                       input_dim=input_dim,
                       activation=self.activation_lstm,
                       dropout_U=self.drop_out,
                       return_sequences=True))
        for i in range(self.lstm_layer-2):
            model.add(LSTM(output_dim=self.output_dim,
                       input_dim=self.output_dim,
                       activation=self.activation_lstm,
                       dropout_U=self.drop_out,
                       return_sequences=True))
        # argument return_sequences should be false in last lstm layer to avoid input dimension incompatibility with dense layer
            model.add(LSTM(output_dim=self.output_dim,
                       input_dim=self.output_dim,
                       activation=self.activation_lstm,
                       dropout_U=self.drop_out))
        for i in range(self.dense_layer-1):
            model.add(Dense(output_dim=self.output_dim,
                        activation=self.activation_last))
            model.add(Dense(output_dim=output_dim,
                        input_dim=self.output_dim,
                        activation=self.activation_last))
        # configure the learning process
            model.compile(loss=self.loss, optimizer=self.optimizer, metrics=['accuracy'])
        # train the model with fixed number of epoches
            model.fit(x=trainX, y=trainY, nb_epoch=self.nb_epoch, batch_size=self.batch_size, validation_data=(testX, testY))
        # store model to json file
            model_json = model.to_json()
        with open(model_path, "w") as json_file:
            json_file.write(model_json)
        # store model weights to hdf5 file
        if model_weight_path:
            if os.path.exists(model_weight_path):
                os.remove(model_weight_path)
            model.save_weights(model_weight_path) # eg: model_weight.h5
        return model

Using TensorFlow backend.


In [111]:

model = NeuralNetwork(output_dim=3, batch_size=1)

(400,)
